In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import missingno as mno
import plotly.offline as pyo 
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
color_pal = sns.color_palette()
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')

import nltk
import re

import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

#Model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
df=pd.read_csv("/kaggle/input/who-suicide-data-1950-2021/combined_processed_data.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.describe(include='object').T

In [ ]:
unique_values = df.nunique()
unique_values

In [ ]:
print(df.isna().sum())
df.isna().sum().plot(kind='bar')
plt.show()
sns.barplot(df.isna())
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Finding duplicate rows
duplicate_rows = df[df.duplicated(keep='first')]

# Number of duplicate rows
num_duplicates = duplicate_rows.shape[0]

# Displaying the duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
duplicate_rows

In [ ]:
# Import the ProfileReport class from the ydata_profiling library
from ydata_profiling import ProfileReport

# Create a comprehensive profile report for the DataFrame 'df'
# This report will contain various statistics, insights, and visualizations about the data
profile = ProfileReport(df)

In [ ]:
profile                     # or this one
profile.to_notebook_iframe() # use this line to show the output

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
columns_to_format = [
    '0 Year', '1-4 Years', '5-9 Years', '10-14 Years', '15-19 Years', 
    '20-24 Years', '25-29 Years', '30-34 Years', '35-39 Years', '40-44 Years', 
    '45-49 Years', '50-54 Years', '55-59 Years', '60-64 Years', '65-69 Years', 
    '70-74 Years', '75-79 Years', '80-84 Years', '85+ Years', 'Unknown Age', 
    'No of Suicides'
]

# Iterate through the selected columns and format the numbers
for column in columns_to_format:
    df[column] = df[column].apply(lambda x: f'{x:.0f}')
    df[column] = pd.to_numeric(df[column])

# Print the DataFrame to see the formatted and numeric columns

In [ ]:
df.head()

In [ ]:
# Correlation:
plt.figure(figsize=(19, 17))
sns.heatmap(df.corr(), cmap="Blues", annot = True)

In [ ]:
import pandas as pd
import plotly.express as px

# Assuming you have a DataFrame named 'df'

# Columns for which you want to create plots
columns_to_plot = ['Region Name', 'Country Name','Sex']

for column in columns_to_plot:
    top10_values = df[column].value_counts()[:10]
    
    # Create a bar plot using Plotly Express
    fig = px.bar(
        y=top10_values.values,
        x=top10_values.index,
        color=top10_values.index,
        color_discrete_sequence=px.colors.sequential.PuBuGn,
        text=top10_values.values,
        title=f'Top 10 {column}',
        template='plotly_white'
    )
    fig.update_layout(
        title_text=f'{column} Distribution',
        height=700,
        xaxis_title=column,
        yaxis_title="Count",
        font=dict(size=17, family="Franklin Gothic")
    )
    
    # Display the plot
    fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Assuming you have a DataFrame named 'df'

# Columns and y-labels for which you want to create plots
columns_to_plot = [
    ('Region Name', 'No of Suicides', 'Suicides by Region Name'),
    ('Country Name', 'Percentage of cause-specific deaths out of total deaths', 'Country Name by Percentage of Deaths'),
    ('Sex','No of Suicides','Sex by No of Suicides')
]

for column, y_label, title in columns_to_plot:
    data = df.groupby(column)[y_label].sum().reset_index()
    
    # Create a line plot using Plotly Express
    fig = px.line(
        data,
        x=column,
        y=y_label,
        title=title,
        height=800
    )
    fig.update_layout(
        xaxis_title=column,
        yaxis_title=y_label,
        font=dict(size=17, family="Franklin Gothic")
    )
    
    # Display the plot
    fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo

# Assuming you have a DataFrame named 'df'

# Columns and y-labels for which you want to create plots
columns_to_plot = [
    ('Region Name', 'Top 10 Region Name by No of Suicides', 'No of Suicides'),
    ('Country Name', 'Top 10 Country Name by No of Suicides', 'No of Suicides')
]

# Initialize Plotly for notebook integration
pyo.init_notebook_mode(connected=True)

for column, title, y_label in columns_to_plot:
    # Calculate the average for the current column and sort
    average_data = df.groupby(column)[y_label].mean().reset_index()
    average_data = average_data.sort_values(by=y_label, ascending=False)
    
    # Select the top 10 items
    top_10_data = average_data.head(10)
    
    # Create a new figure using Plotly
    fig = go.Figure()
    
    # Add a bar trace to the figure
    fig.add_trace(go.Bar(
        x=top_10_data[column],
        y=top_10_data[y_label]
    ))
    
    # Update the layout and appearance of the plot
    fig.update_layout(
        title=title,
        xaxis_title=column,
        yaxis_title=y_label,
        template='plotly_white',
        font=dict(color='black'),
        height=800
    )
    
    # Display the plot
    pyo.iplot(fig)


In [ ]:
# x-axis: "work_year", y-axis: "salary_in_usd", color-coded by "experience_level",
# and create a notched box plot
fig = px.box(df, x="Region Name", y="No of Suicides", color="Sex", notched=True)

# Update the layout of the plot
fig.update_layout(
    title_text='No of Suicides Box Distribution',  # Set the title of the plot
    height=500,  # Set the height of the plot
    xaxis_title="Region Name",  # Label for the x-axis
    yaxis_title="No of Suicides",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Columns and titles for which you want to create sunburst charts
columns_to_plot = [
    ('Sex', 'Distribution of Sex'),
    ('Country Name', 'Distribution of Country Name'),
    ('Region Name','Distribution of Region Name')
]

for column, title in columns_to_plot:
    # Create a sunburst chart using Plotly Express
    fig = px.sunburst(df, path=[column], color_discrete_sequence=px.colors.qualitative.Set2)
    fig.update_layout(title_text=title, height=500)
    
    # Display the plot
    fig.show()

In [ ]:
fig = px.pie(df, values="No of Suicides", names="Region Name",
             color_discrete_sequence=px.colors.sequential.RdBu,
             opacity=0.7, hole=0.5)
fig.show()

In [ ]:

# reading the database
plot = go.Figure(data=[
    go.Scatter(
        x=df['Region Name'],
        y=df['No of Suicides'],
        mode='markers',
        marker=dict(
            color=[{'Male': 'blue', 'Female': 'pink', 'Unknown': 'yellow'}[gender] for gender in df['Sex']]
        )
    )
])
 
# Add dropdown
plot.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["type", "scatter"],
                    label="Scatter Plot",
                    method="restyle",
                ),
               dict(
                    args=["type", "bar"],
                    label="Bar Chart",
                    method="restyle",
                )
            ]),
            direction="down",
        ),
    ]
)
 
plot.show()


In [ ]:
import pandas as pd
import plotly.express as px
fig = px.scatter(df, x="No of Suicides", y="Region Name", size="No of Suicides", color="Sex",
                 hover_name="Country Name", log_x=True, size_max=60)
fig.show()

In [ ]:
fig = px.box(df, x="Region Name", y="No of Suicides")
 
# showing the plot
fig.show()

In [ ]:
fig = px.scatter_matrix(df, height=1800, width=2000)
fig.show()

In [ ]:
# Box Plot
# Prepare data for the box plot
hist_data = [df['No of Suicides']]
group_labels = ['No of Suicides']

# Create the box plot using Plotly Express
fig1 = px.box(y=df['No of Suicides'], template='plotly_white', title='No of Suicides (BoxPlot)')

# Customize the layout of the box plot
fig1.update_layout(font=dict(size=17, family="Franklin Gothic"))

# Display the box plot
fig1.show()

# Dist Plot
# Prepare data for the distribution plot
hist_data = [df['No of Suicides']]
group_labels = ['No of Suicides']

# Create the distribution plot using Plotly Figure Factory
fig2 = ff.create_distplot(hist_data, group_labels, show_hist=False)
fig2.layout.template = 'plotly_white'

# Customize the layout of the distribution plot
fig2.update_layout(title='PrNo of Suicides (DistPlot)', font=dict(size=17, family="Franklin Gothic"))

# Display the distribution plot
fig2.show()

In [ ]:
df['Region Name'].unique()

In [ ]:
top_10=pd.DataFrame(df.groupby("Region Name")["No of Suicides"].sum().sort_values(ascending=False).head(10)).reset_index()
top_10

In [ ]:
top_10=pd.DataFrame(df.groupby("Country Name")["No of Suicides"].sum().sort_values(ascending=False).head(10)).reset_index()
top_10

In [ ]:
import plotly.express as px
import pandas as pd
# Assuming you have a DataFrame named 'df'

# Group by 'Sex' and 'Year', and calculate the sum of 'No of Suicides'
sex_year = df.groupby(["Sex", "Year"])["No of Suicides"].sum().reset_index()

# Create a bar plot for each type in 'Sex'
fig = px.bar(
    sex_year,
    x="Year",
    y="No of Suicides",
    color="Sex",
    facet_col="Sex",
    labels={'Year': 'Year', 'No of Suicides': 'Number of Suicides'},
    template='plotly',
    height=400
)

# Rotate x-axis labels for better readability
fig.update_xaxes(tickangle=90)

# Show the plots
fig.show()


In [ ]:
# Select columns with object (categorical) data types
num_cols = df.select_dtypes(include='object').columns.tolist()

# Initialize the LabelEncoder
le = LabelEncoder()

# Apply Label Encoding to the selected numerical columns
for x in num_cols:  
    df[x] = le.fit_transform(df[x])

# Now, your categorical columns (excluding column 0) have been converted to numerical values
df.head()

In [ ]:
Corr_Matrix = df.corr()

# Set up the figure and plot the heatmap
plt.figure(figsize=(30, 30))
sns.heatmap(Corr_Matrix, annot=True, cmap='BuPu', center=0)
plt.show()

In [ ]:
print('Top 5 Most Positively Correlated to the To salary_in_usd')
Corr_Matrix['No of Suicides'].sort_values(ascending=False).head(5)

In [ ]:
print('Top 5 Most Positively Correlated to salary_in_usd ')
Corr_Matrix['No of Suicides'].sort_values(ascending=True).head(5)

In [ ]:
X = df.drop(columns=['No of Suicides'])
y = df['No of Suicides']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['Actual_salary'] = y_test
    submit['Predict_salary'] = y_pred
    submit = submit.reset_index()
    r2 = r2_score(y_test, y_pred)
    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(submit.head(5))

    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")